In [1]:
# imports
import os
import random
import warnings
from pathlib import Path

import ccdproc
from astropy import units
from astropy.nddata import CCDData
from astropy.utils.exceptions import AstropyWarning
from rich import print, pretty
from tqdm import trange, tqdm

In [2]:
# configurations
# supporess fits warnings
warnings.filterwarnings("ignore", category=AstropyWarning, append=True)

# configure rich
pretty.install()

# configure tqdm
bar_format = {"unit_scale": True,
              "unit": "Files",
              "colour": "green",
              "ascii": True,
              "ncols": 90}

# directory definitions
initial_data_dir = Path("AstronomyResearchData/20230525/")
fixed_data_dir = Path("AstronomyResearchData/PreProcessData_20230525")

# make the final dir if it doesnt exist
os.makedirs(fixed_data_dir, exist_ok=True)

In [3]:
# collect all files from the initial data dir and read them in
# save them to a list of dictionaries with the raw file and
# just the fits headers seperated

# fits_files = []
# for file in tqdm(ccdproc.ImageFileCollection(initial_data_dir).files_filtered(include_path=True), desc="Loading Fits Files", **bar_format):
#     fits_file = CCDData.read(file)
#     fits_files.append({
#         "file": file,
#         "fits": fits_file
#     })

fits_files = [{"file": file, "fits": CCDData.read(file)} for file in tqdm(ccdproc.ImageFileCollection(initial_data_dir).files_filtered(include_path=True), desc="Loading Fits Files", **bar_format)]

Loading Fits Files: 100%|#############################| 326/326 [00:19<00:00, 16.9Files/s]


In [4]:
# sort files and display the files that remain
bias_files = []
dark_files = []
sky_files = []
light_files = []
remainder_files = []

for file in tqdm(fits_files, desc="Sorting", **bar_format):
    ftype = file["fits"].meta["IMTYPE"]
    ffilter = file["fits"].meta["FILTER"]

    # remove files with filters I did not use
    if ffilter not in ["Bessel R", "Bessel B", "Bessel V"] and ftype not in ["Bias", "Dark"]: continue

    # sort away files by type
    if ftype == "Sky" or "Flat" in file["file"]:
        sky_files.append(file)
        continue
    elif "Dark" in os.path.basename(file["file"]):
        dark_files.append(file)
        continue
    elif ftype == "Bias":
        bias_files.append(file)
        continue
    elif ftype == "Light" and file["fits"].meta["OBJECT"] == "":
        light_files.append(file)
        continue

    # explicit catch for a particular broken file
    if ftype == "Dark" and file["fits"].meta["OBJECT"] == "Python":
        light_files.append(file)
        continue

    # put all unsorted files in remainder_files
    remainder_files.append(file)

# output file sorting
print(f"Sorting yielded {len(remainder_files)} unsorted files.")
print([f["file"] for f in remainder_files])

Sorting: 100%|#######################################| 326/326 [00:00<00:00, 8.42kFiles/s]


Sorting yielded 3 unsorted files.

[
    'AstronomyResearchData/20230525/2023gfo_besselV_180s_001.fits',
    'AstronomyResearchData/20230525/2023gfo_besselV_180s_002.fits',
    'AstronomyResearchData/20230525/2023gfo_besselV_180s_003.fits'
]

In [5]:
# create master bias (no further sorting needed)
# combine fits
master_bias = ccdproc.combine([f["fits"] for f in bias_files], method="median")

# add master flat
master_bias.meta["MASTERIM"] = True
master_bias.meta["IMTYPE"] = "Bias"
master_bias.meta["OBJECT"] = ""

# save to final dir
master_bias.write(Path(fixed_data_dir, "MasterBias.fits"), overwrite=True)

In [6]:
# create master darks
dark_times = set([f["fits"].meta["EXPTIME"] for f in dark_files])
master_darks = {}

# init live pbar
pbar = tqdm(dark_times, desc="Making Darks", **bar_format)

# sort through fits by dark times, ineficient but who cares
for dark_time in pbar:
    # update pbar
    pbar.set_description(f"Making {str(int(dark_time))}s Dark")

    # get only files for this time
    timed_dark_files = [f["fits"] for f in dark_files if f["fits"].meta["EXPTIME"] == dark_time and f["fits"].meta["IMTYPE"] != "Light"]

    # create master dark for this time
    master_dark = ccdproc.combine([f for f in timed_dark_files], method="median")

    # add master flag
    master_dark.meta["MASTERIM"] = True
    master_dark.meta["IMTYPE"] = "Dark"
    master_dark.meta["EXPTIME"] = dark_time
    master_dark.meta["OBJECT"] = ""

    # save master dark for this time
    master_dark.write(Path(fixed_data_dir, f"MasterDark_{str(int(dark_time))}.fits"), overwrite=True)
    master_darks[dark_time] = master_dark


Making 30s Dark: 100%|##############################| 14.0/14.0 [00:41<00:00, 2.99s/Files]


In [7]:
# create master flats
filters = set([f["fits"].meta["FILTER"] for f in sky_files])
master_sky = {}

# init live pbar
pbar = tqdm(filters, desc="Making Flats", **bar_format)

# sort through skys by filter, ineficient but again who cares
for sky_filter in pbar:
    # update pbar
    pbar.set_description(f"Making {sky_filter} Flat")

    # get only files for this filter
    filtered_sky = [f["fits"] for f in sky_files if f["fits"].meta["filter"] == sky_filter]

    # create master flat
    master_flat = ccdproc.combine(filtered_sky, method="median")

    # add master flag
    master_flat.meta["MATERIM"] = True
    master_flat.meta["IMTYPE"] = "Sky"
    master_flat.meta["OBJECT"] = ""

    # save master flat for this filter
    text_filter = str(sky_filter).replace(" ", "-")
    master_flat.write(Path(fixed_data_dir, f"MasterFlat_{text_filter}.fits"), overwrite=True)
    master_sky[sky_filter] = master_flat

Making Bessel R Flat: 100%|#########################| 3.00/3.00 [00:10<00:00, 3.45s/Files]


In [8]:
# reduce lights
# create live pbar
new_bar_format = bar_format.copy()
new_bar_format["ncols"] = 30
pbar = tqdm([(f["file"], f["fits"]) for f in light_files], desc="Reducing Lights", **bar_format)

# parse through lights and apply calibrations
for fname, lfits in pbar:
    # update pbar
    pbar.set_description(f"Reducing {os.path.basename(fname)}")

    # get params from lights
    exp_time = lfits.meta["EXPTIME"]
    fit_filter = lfits.meta["FILTER"]

    # bias subtract
    reduced_fit = ccdproc.subtract_bias(lfits, master_bias)

    # check for dark subtraction
    reduced_fit = ccdproc.subtract_dark(reduced_fit,
                                        master_darks[exp_time],
                                        dark_exposure=exp_time*units.s,
                                        data_exposure=exp_time*units.s)

    # flat divide
    reduced_fit = ccdproc.flat_correct(reduced_fit, master_sky[fit_filter])

    # update header
    params = fname.split("_")

    # get obj from name and replace fits header
    obj = params[1]
    reduced_fit.meta["OBJECT"] = obj

    # ensure exptime and filter passed through
    reduced_fit.meta["EXPTIME"] = exp_time
    reduced_fit.meta["FILTER"] = fit_filter
    text_filter = fit_filter.replace(" ", "-")

    # set an initial value for num, will be updated later
    num = 0
    fname = f"{obj}_{exp_time}_{fit_filter}_{num}.fits"
    while Path(fixed_data_dir, fname).is_file():
        num += 1
        fname = f"{obj}_{exp_time}_{fit_filter}_{num}.fits"

    # save file
    reduced_fit.write(Path(fixed_data_dir, fname), overwrite=True)

Reducing _W-UMa_Bessel-V_15_010_PY.fits: 100%|########| 170/170 [01:19<00:00, 2.15Files/s]
